In [1]:
from numpy import *
from scipy.sparse import *

In [2]:
trackArtist=load_npz("fucking_ordered_track_artist.npz")
trackAlbum=load_npz("fucking_ordered_track_album.npz")

In [3]:
simByArtist=trackArtist*trackArtist.T
simByAlbum=trackAlbum*trackAlbum.T

In [4]:
simByArtist,simByAlbum

(<100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 2590190 stored elements in Compressed Sparse Row format>,
 <100000x100000 sparse matrix of type '<class 'numpy.int64'>'
 	with 394150 stored elements in Compressed Sparse Row format>)

In [51]:
S=simByArtist+0.5*simByAlbum
S

<100000x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 2592296 stored elements in Compressed Sparse Row format>

In [52]:
R=load_npz("all_playlist_with_tracks_URM.npz")
R

<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 1040522 stored elements in Compressed Sparse Column format>

In [87]:
RR=R*S
RR

<45649x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 21135411 stored elements in Compressed Sparse Column format>

In [42]:
allPlaylist=unique(genfromtxt("train_final.csv",usecols=0,dtype=int32))[1:]

In [57]:
allPlaylist

array([    7569,     7614,     7641, ..., 11765405, 11766293, 11766362], dtype=int32)

In [62]:
allTracks=unique(load("ordered_tracks.npy").astype(int32))
allTracks,allTracks.size

(array([    252,     360,     394, ..., 5007354, 5014702, 5018274], dtype=int32),
 100000)

In [79]:
max([max(x.data) for x in RR])

144.5

In [88]:
RR=RR-200*R
RR

<45649x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 21135411 stored elements in Compressed Sparse Column format>

In [99]:
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
targetTrack.size

32195

In [125]:
maskT=list(map(lambda x:x in targetTrack,allTracks))
len(maskT)

100000

In [121]:
targetPlaylist=unique(genfromtxt("target_playlists.csv",skip_header=1,dtype=int32))
targetPlaylist.size

10000

In [124]:
maskP=list(map(lambda x:x in targetPlaylist,allPlaylist))
len(maskP)

45649

In [151]:
with open("noTags.csv","w") as f:
    f.write("playlist_id,track_ids\n")
    for i,sims in enumerate(RR):
        if(maskP[i]):
            f.write(str(allPlaylist[i])+",")
            suggestions=allTracks[flip(argsort(sims.multiply(mask).todense()),1).take(range(5))]
            for t in suggestions[0]:
                f.write(str(t)+" ")
            f.write("\n")